In [1]:
import requests
import pandas as pd
from vietnamadminunits.parser.utils import unicode_normalize, correct_typos, uppercase_first_letters

import warnings
warnings.filterwarnings('ignore')

import re

In [3]:
res = requests.get('https://production.cas.so/address-kit/2025-07-01/communes')
data = res.json()['communes']

In [4]:
df_code = pd.DataFrame(data)

In [5]:
cols = {
    'code': 'wardCode',
    'name': 'ward',
    'provinceName': 'province',
    'provinceCode': 'provinceCode',
}
df_code.rename(columns=cols, inplace=True)

In [8]:
df_code_province = df_code[['provinceCode', 'province']].drop_duplicates()

In [9]:
df = pd.read_csv('/Users/hieutran/Documents/GitHub/personal/sapnhap/data/processed/2025_34-province-3221-ward_with_location.csv')

In [10]:
df.loc[df['province']=='Thủ đô Hà Nội', 'province'] = 'Thành phố Hà Nội'

In [11]:
df_province = df[['province']].drop_duplicates()

In [12]:
df_province_code = pd.merge(df_province, df_code_province, on='province', how='left')

In [13]:
df_province_code

,province,provinceCode
0,Thành phố Hà Nội,01
1,Tỉnh Tuyên Quang,08
2,Tỉnh Cao Bằng,04
3,Tỉnh Thái Nguyên,19
4,Tỉnh Lào Cai,15
5,Tỉnh Điện Biên,11
6,Tỉnh Lai Châu,12
7,Tỉnh Sơn La,14
8,Tỉnh Phú Thọ,25
9,Tỉnh Lạng Sơn,20


In [ ]:
df_code['ward'] = df_code['ward'].str.replace('\n', '')

In [14]:
df_code['ward'] = df_code['ward'].apply(unicode_normalize)
df['ward'] = df['ward'].apply(unicode_normalize)

In [15]:
df_code['ward'] = df_code['ward'].apply(correct_typos)
df['ward'] = df['ward'].apply(correct_typos)

In [16]:
df_code['ward'] = df_code['ward'].apply(uppercase_first_letters)
df['ward'] = df['ward'].apply(uppercase_first_letters)

In [17]:
df.loc[(df['province']=='Thành phố Hải Phòng') & (df['ward']=='Xã Việt Hòa'), 'ward'] = 'Phường Việt Hòa'
df.loc[(df['province']=='Thành phố Hải Phòng') & (df['ward']=='Phường Vĩnh Hòa'), 'ward'] = 'Xã Vĩnh Hòa'
df.loc[(df['province']=='Thành phố Cần Thơ') & (df['ward']=='Xã Tân Lộc'), 'ward'] = 'Phường Tân Lộc'
df.loc[(df['province']=='Thành phố Cần Thơ') & (df['ward']=='Phường Phong Điền'), 'ward'] = 'Xã Phong Điền'

In [18]:
df_with_code = pd.merge(df, df_code, on=['province', 'ward'], how='left')

In [19]:
df_with_code

,province,ward,provinceShort,wardShort,wardType,provinceCode_x,provinceLat,provinceLon,wardCode_x,wardLat,wardLon,wardAreaKm2,wardCode_y,provinceCode_y
0,Thành phố Hà Nội,Phường Hồng Hà,Hà Nội,Hồng Hà,Phường,1,21.0001,105.698,42.0,21.05670,105.845,15.09,00097,01
1,Thành phố Hà Nội,Phường Ba Đình,Hà Nội,Ba Đình,Phường,1,21.0001,105.698,3.0,21.03860,105.838,2.97,00004,01
2,Thành phố Hà Nội,Phường Ngọc Hà,Hà Nội,Ngọc Hà,Phường,1,21.0001,105.698,61.0,21.03810,105.816,2.68,00008,01
3,Thành phố Hà Nội,Phường Giảng Võ,Hà Nội,Giảng Võ,Phường,1,21.0001,105.698,30.0,21.02750,105.814,2.60,00025,01
4,Thành phố Hà Nội,Phường Hoàn Kiếm,Hà Nội,Hoàn Kiếm,Phường,1,21.0001,105.698,39.0,21.03200,105.850,1.91,00070,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3316,Tỉnh Cà Mau,Xã Phú Tân,Cà Mau,Phú Tân,Xã,34,9.1362,105.182,3331.0,8.92852,104.846,101.70,32218,96
3317,Tỉnh Cà Mau,Xã Nguyễn Việt Khái,Cà Mau,Nguyễn Việt Khái,Xã,34,9.1362,105.182,3324.0,8.86914,104.930,129.90,32227,96
3318,Tỉnh Cà Mau,Xã Tân Ân,Cà Mau,Tân Ân,Xã,34,9.1362,105.182,3337.0,8.72769,105.078,218.30,32236,96
3319,Tỉnh Cà Mau,Xã Phan Ngọc Hiển,Cà Mau,Phan Ngọc Hiển,Xã,34,9.1362,105.182,3327.0,8.64616,104.943,237.70,32244,96


In [20]:
df_with_code[df_with_code['wardCode_y'].isna()]

,province,ward,provinceShort,wardShort,wardType,provinceCode_x,provinceLat,provinceLon,wardCode_x,wardLat,wardLon,wardAreaKm2,wardCode_y,provinceCode_y
